# Partisan family analysis

The `process_county` function below groups voter registration records by last name, address and party to calculate "split-voting families". Please be mindful of the following definitions:

`Family`: A combination of an address and a last name where multiple voters with the same last name registered at the same address voted in a March 2018 primary

`Split-voting family`: `Family` where at least one member voted in the March 2018 Republican primary and at least one member voted in the Democratic primary

`Split voting families percentage`: Number of `Split-voting families` divided by the number of `Families`

In [1]:
import pandas as pd
from IPython.display import display, HTML

In [2]:
def process_county(df):
    '''
    Receives a data frame as input and displays tables for 
    '''
    
    # Initialize an array to display later
    data_to_display = []
    
    # Only deal with voters who participated in the March 2018 primary
    # The `party_primary` column reflect the March 2018 election

    df = df.assign(
        voted = lambda x: x.primary_party.isnull() == False
    ).query('voted')

    # Make a dataframe of "Families" that voted in the primary
    df_family = df.set_index([
        'address',
        'last_name'
    ])[['primary_party']].assign(
        # Family size is the number of voters with the same last name
        # registered at an address
        family_size = df.groupby([
            'address',
            'last_name'
        ]).size()
    # Filter for "Families" - multiple of same last name
    ).query('family_size > 1').assign(
        # Make a combination address-last name field for counting purposes
        address_family = lambda x: x.index.get_level_values(0) + x.index.get_level_values(1)
    )

    data_to_display.append({
        'Metric': 'Families',
        'Value': '{:,}'.format(len(
            # Count the distinct address-families
            df_family.reset_index().address_family.unique()
        ))
    })

    # Make a dataframe of "Split-voting families" - "Families" that voted in both primaries
    df_split_family = df_family.join(
        df.groupby([
            'address',
            'last_name'
        ]).agg({
            # Count the different parties the family voted in
            'primary_party': lambda x: len(x.unique())
        }).rename(
            columns = {
                'primary_party': 'name_party_count'
            }
        # Filter for "Families" that voted in both primaries ("R" and "D")
        ).query('name_party_count == 2'),
        how='inner'
    ).join(
        # Get the other columns from the full dataframe
        df.set_index([
            'address',
            'last_name'
        ]),
        lsuffix='_',
        rsuffix=''
    ).reset_index().assign(
        address_family = lambda x: x.address + x.last_name
    )

    df_split_family_total = len(
        df_split_family.address_family.unique()
    )

    data_to_display.append({
        'Metric': 'Split-vote families',
        'Value': '{:,}'.format(
            df_split_family_total
        )
    })

    data_to_display.append({
        'Metric': 'Split-vote percentage',
        'Value': '{:.2%}'.format(
            df_split_family_total / len(
                df_family.address_family.unique()
            )
        )
    })
    
    display(
        pd.DataFrame(data_to_display).set_index('Metric')
    )

    display(
        HTML(
            '<p>Gender breakdown of voters in split-voting families</p>'
        )
    )
    display(
        pd.crosstab(
            df_split_family.query(
                'sex == "F" | sex == "M"'
            )['primary_party'],
            df_split_family.query(
                'sex == "F" | sex == "M"'
            ).sex
        ).apply(
            lambda x: ['{:.2%}'.format(a) for a in x / sum(x)], axis=1
        )
    )

#### Travis County

In [3]:
process_county(
    pd.read_csv(
        'voter-reg-data/VoterRegOpenDataWithHistory.csv',
        encoding='latin-1',
        dtype='str'
    ).assign(
        address = lambda x: (x.MLADD1.astype(str) + ', ' + x.MLCITY.astype(str) + ', ' + x.MZIPCD.astype(str)).str.strip(),
        primary_party = lambda x: x.P18PARTY,
        last_name = lambda x: x.LSTNAM,
        sex = lambda x: x.GENDER
    )
)

,Value
Metric,
Families,"23,283"
Split-vote families,"1,777"
Split-vote percentage,7.63%


sex,F,M
primary_party,,
DEMO,69.89%,30.11%
REPB,28.50%,71.50%


### Williamson County

In [4]:
process_county(
    pd.read_excel(
        'voter-reg-data/20180507_CWDOB.xlsx',
        dtype='str'
    ).assign(
        age = lambda x: x.birthdate.apply(lambda z: (pd.datetime.now().year - pd.to_datetime(z).year)),
        address = lambda x: (x.streetnumber.astype(str) + ' ' +
                             x.streetname + ' ' + x.streettype + ', ' +
                             x.city.astype(str) + ', ' + x['zip'].astype(str)).str.strip(),
        last_name = lambda x: x.lastname,
        primary_party = lambda x: x['party_code2'].replace(
            'nan', pd.np.nan
        )
    )
)

,Value
Metric,
Families,"16,131"
Split-vote families,896
Split-vote percentage,5.55%


sex,F,M
primary_party,,
D,71.88%,28.12%
R,30.76%,69.24%


#### Hays County

In [5]:
process_county(
    pd.read_excel(
        'voter-reg-data/Hays-May7.xlsx',
        dtype='str'
    ).assign(
        last_name = lambda x: x['FULL NAME'].str.split(
            ','
        ).str.get(0),
        age = lambda x: x.birthdate.apply(lambda z: (pd.datetime.now().year - pd.to_datetime(z).year)),
        address = lambda x: (x.streetnumber.astype(str) + ' ' +
                             x.streetname + ' ' + x.streettype + ', ' +
                             x.city.astype(str) + ', ' + x['zip'].astype(str)).str.strip(),
        primary_party = lambda x: x['party_code1'].replace('nan', pd.np.nan)
    )
)

,Value
Metric,
Families,"5,731"
Split-vote families,377
Split-vote percentage,6.58%


sex,F,M
primary_party,,
D,70.36%,29.64%
R,30.36%,69.64%
